# Notebook 06: Dynamical Systems Analysis

## Understanding Neural Networks as Dynamical Systems

This notebook explores how to analyze neural networks through the lens of **dynamical systems theory**. Instead of viewing networks as static function approximators, we treat them as systems that evolve over time, with rich temporal dynamics, attractors, and stability properties.

### Why Dynamical Systems Matter for Mechanistic Interpretability

1. **Recurrent Networks are Dynamical Systems**: RNNs, LSTMs, and transformers with recurrence naturally evolve over time
2. **Attractors Explain Computation**: Fixed points and limit cycles correspond to stable computational states
3. **Stability Reveals Robustness**: Lyapunov exponents tell us if networks are chaotic or stable
4. **Dimensionality Reveals Complexity**: Intrinsic dimensionality shows true degrees of freedom
5. **Controllability Enables Steering**: Understanding control allows us to guide network behavior

### What You'll Learn

1. **Koopman Operator Theory**: Linearizing nonlinear dynamics for analysis
2. **Dynamic Mode Decomposition (DMD)**: Data-driven approach to finding dynamic modes
3. **Lyapunov Exponents**: Measuring chaos and stability in neural dynamics
4. **Fixed Point Analysis**: Finding and characterizing stable states
5. **Intrinsic Dimensionality**: Measuring true representational complexity
6. **Controllability Analysis**: Understanding how to steer network dynamics

### References

- Kutz et al. (2016): *Dynamic Mode Decomposition: Data-Driven Modeling of Complex Systems*
- Brunton & Kutz (2019): *Data-Driven Science and Engineering*
- Sussillo & Barak (2013): *Opening the Black Box: Low-Dimensional Dynamics in High-Dimensional RNNs*
- Maheswaranathan et al. (2019): *Universality and individuality in neural dynamics*
- Jazayeri & Afraz (2017): *Navigating the neural space in search of the neural code*

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.optimize import fsolve
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import torch
import torch.nn as nn

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")

## Part 1: Koopman Operator Theory and Dynamic Mode Decomposition

### The Koopman Operator: Linearizing Nonlinear Dynamics

The **Koopman operator** is a powerful mathematical tool that allows us to analyze nonlinear dynamical systems using linear techniques. Here's the key insight:

**Nonlinear system**:
```
x(t+1) = F(x(t))  # Nonlinear evolution
```

**Koopman operator** acts on *observables* g(x):
```
K g(x) = g(F(x))  # Linear operator on infinite-dimensional space
```

The beauty: Although F is nonlinear, K is *linear* (but infinite-dimensional).

**Dynamic Mode Decomposition (DMD)** approximates K from data:
- Collect snapshots: X = [x(0), x(1), ..., x(T-1)]
- Collect next states: X' = [x(1), x(2), ..., x(T)]
- Find best-fit linear operator: X' ≈ A X
- Eigendecomposition of A gives dynamic modes and growth rates

### Why This Matters for Neural Networks

1. **RNN Dynamics**: Understand how hidden states evolve over time
2. **Mode Decomposition**: Identify dominant patterns in neural activity
3. **Forecasting**: Predict future states from current observations
4. **Control**: Design inputs to guide dynamics to desired states

In [ ]:
class DynamicModeDecomposition:
    """
    Dynamic Mode Decomposition (DMD) for analyzing temporal dynamics.
    
    DMD finds the best-fit linear operator that advances dynamics:
        X' = A X
    
    The eigendecomposition of A reveals:
    - Dynamic modes (spatial patterns)
    - Eigenvalues (growth/decay rates and frequencies)
    - Mode amplitudes (importance of each mode)
    """
    
    def __init__(self, rank=None):
        """
        Args:
            rank: Rank for truncated SVD. If None, use full rank.
        """
        self.rank = rank
        self.modes = None
        self.eigenvalues = None
        self.amplitudes = None
    
    def fit(self, X, dt=1.0):
        """
        Fit DMD to trajectory data.
        
        Args:
            X: Data matrix of shape (n_features, n_timesteps)
               Columns are snapshots at different times
            dt: Time step between snapshots
        """
        # Split into current and next states
        X1 = X[:, :-1]  # x(0), x(1), ..., x(T-1)
        X2 = X[:, 1:]   # x(1), x(2), ..., x(T)
        
        # Compute SVD of X1
        U, S, Vt = np.linalg.svd(X1, full_matrices=False)
        
        # Truncate if rank specified
        if self.rank is not None:
            U = U[:, :self.rank]
            S = S[:self.rank]
            Vt = Vt[:self.rank, :]
        
        # Build reduced DMD operator: Atilde = U^T @ X2 @ V @ S^{-1}
        Atilde = U.T @ X2 @ Vt.T @ np.diag(1/S)
        
        # Eigendecomposition of reduced operator
        eigvals, eigvecs = np.linalg.eig(Atilde)
        
        # Reconstruct full modes: Phi = X2 @ V @ S^{-1} @ W
        self.modes = X2 @ Vt.T @ np.diag(1/S) @ eigvecs
        
        # Continuous-time eigenvalues
        self.eigenvalues = np.log(eigvals) / dt
        
        # Compute mode amplitudes from initial condition
        self.amplitudes = np.linalg.lstsq(self.modes, X[:, 0], rcond=None)[0]
        
        return self
    
    def predict(self, t):
        """
        Predict state at time t using DMD reconstruction.
        
        x(t) = sum_k [ b_k * phi_k * exp(lambda_k * t) ]
        
        Args:
            t: Time or array of times
        
        Returns:
            Predicted state(s)
        """
        t = np.atleast_1d(t)
        
        # Compute time dynamics: exp(lambda * t)
        time_dynamics = np.exp(np.outer(self.eigenvalues, t))
        
        # Weighted sum: Phi @ diag(b) @ time_dynamics
        predictions = self.modes @ (self.amplitudes[:, np.newaxis] * time_dynamics)
        
        return predictions.real
    
    def get_mode_properties(self):
        """
        Extract interpretable properties of each mode.
        
        Returns:
            Dictionary with mode properties:
            - growth_rates: Real part of eigenvalues (stability)
            - frequencies: Imaginary part of eigenvalues (oscillation)
            - amplitudes: Mode amplitudes (importance)
        """
        return {
            'growth_rates': self.eigenvalues.real,
            'frequencies': self.eigenvalues.imag,
            'amplitudes': np.abs(self.amplitudes),
            'modes': self.modes
        }

print("DMD class implemented!")

In [ ]:
# Create synthetic RNN-like dynamics for demonstration
def generate_rnn_dynamics(n_timesteps=200, n_neurons=50):
    """
    Generate synthetic recurrent neural network dynamics.
    Creates a system with multiple oscillatory and decaying modes.
    """
    t = np.linspace(0, 10, n_timesteps)
    
    # Create multiple dynamic modes
    # Mode 1: Slow oscillation (frequency = 1 Hz, stable)
    mode1 = np.outer(np.random.randn(n_neurons), np.sin(2*np.pi*1*t) * np.exp(-0.1*t))
    
    # Mode 2: Fast oscillation (frequency = 3 Hz, decaying)
    mode2 = np.outer(np.random.randn(n_neurons), np.cos(2*np.pi*3*t) * np.exp(-0.3*t))
    
    # Mode 3: Exponential decay
    mode3 = np.outer(np.random.randn(n_neurons), np.exp(-0.5*t))
    
    # Combine modes
    X = mode1 + 0.5*mode2 + 0.3*mode3
    
    # Add small noise
    X += 0.1 * np.random.randn(*X.shape)
    
    return X, t

# Generate data
X, t = generate_rnn_dynamics(n_timesteps=200, n_neurons=50)

print(f"Generated dynamics: {X.shape[0]} neurons, {X.shape[1]} timesteps")
print(f"Time range: {t[0]:.2f} to {t[-1]:.2f} seconds")

In [ ]:
# Fit DMD to the data
dmd = DynamicModeDecomposition(rank=10)  # Use top 10 modes
dmd.fit(X, dt=t[1]-t[0])

# Get mode properties
props = dmd.get_mode_properties()

print("\nDMD Analysis Results:")
print("="*50)
for i in range(min(5, len(props['growth_rates']))):
    print(f"\nMode {i+1}:")
    print(f"  Growth rate: {props['growth_rates'][i]:.4f} (negative = decaying)")
    print(f"  Frequency: {props['frequencies'][i]:.4f} Hz")
    print(f"  Amplitude: {props['amplitudes'][i]:.4f} (importance)")

In [ ]:
# Visualize DMD results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Original dynamics (first 3 neurons)
ax = axes[0, 0]
for i in range(3):
    ax.plot(t, X[i, :], label=f'Neuron {i+1}', alpha=0.7)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Activity')
ax.set_title('Original Neural Dynamics (Sample Neurons)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: DMD reconstruction
ax = axes[0, 1]
X_pred = dmd.predict(t)
for i in range(3):
    ax.plot(t, X_pred[i, :], '--', label=f'Neuron {i+1} (DMD)', alpha=0.7)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Activity')
ax.set_title('DMD Reconstruction')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Eigenvalue spectrum
ax = axes[1, 0]
scatter = ax.scatter(props['growth_rates'], props['frequencies'], 
                    s=props['amplitudes']*1000, 
                    c=props['amplitudes'], cmap='viridis',
                    alpha=0.6, edgecolors='black')
ax.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Stability boundary')
ax.set_xlabel('Growth Rate (Real part)')
ax.set_ylabel('Frequency (Imaginary part, Hz)')
ax.set_title('DMD Eigenvalue Spectrum\n(size = amplitude)')
ax.grid(True, alpha=0.3)
ax.legend()
plt.colorbar(scatter, ax=ax, label='Amplitude')

# Plot 4: Mode amplitudes
ax = axes[1, 1]
mode_indices = np.arange(len(props['amplitudes']))
colors = ['red' if gr > 0 else 'blue' for gr in props['growth_rates']]
ax.bar(mode_indices, props['amplitudes'], color=colors, alpha=0.6)
ax.set_xlabel('Mode Index')
ax.set_ylabel('Amplitude')
ax.set_title('Mode Amplitudes\n(red=growing, blue=decaying)')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Growth rate < 0: Mode decays over time (stable)")
print("- Growth rate > 0: Mode grows over time (unstable)")
print("- Frequency ≠ 0: Mode oscillates")
print("- Amplitude: Importance of mode to overall dynamics")

## Part 2: Lyapunov Exponents - Measuring Chaos and Stability

### What are Lyapunov Exponents?

**Lyapunov exponents** measure how fast nearby trajectories diverge or converge in a dynamical system.

**The largest Lyapunov exponent (λ_max)**:
- λ < 0: System is stable (trajectories converge)
- λ ≈ 0: System is marginally stable
- λ > 0: System is chaotic (trajectories diverge exponentially)

**Mathematical definition**:
```
λ = lim_{t→∞} (1/t) * log(||δx(t)|| / ||δx(0)||)
```
where δx(t) is a small perturbation that evolves over time.

### Why This Matters for Neural Networks

1. **Robustness**: Negative Lyapunov exponents mean networks are robust to noise
2. **Edge of Chaos**: Many successful RNNs operate near λ ≈ 0 (critical regime)
3. **Generalization**: Chaotic networks (λ > 0) may overfit and be unpredictable
4. **Information Processing**: Moderate complexity (near-critical) is optimal for computation

### Computing Lyapunov Exponents from Data

For a discrete-time system x(t+1) = F(x(t)):
1. Start with initial state x(0)
2. Add small perturbation: x_perturbed(0) = x(0) + δ
3. Evolve both trajectories: x(t) and x_perturbed(t)
4. Periodically measure divergence and renormalize
5. Average log growth rate over time

In [ ]:
class LyapunovExponentEstimator:
    """
    Estimate largest Lyapunov exponent from time series data.
    
    Uses the method of tracking divergence of nearby trajectories.
    """
    
    def __init__(self, dt=1.0, perturbation_size=1e-8):
        """
        Args:
            dt: Time step
            perturbation_size: Size of initial perturbation
        """
        self.dt = dt
        self.perturbation_size = perturbation_size
    
    def estimate_from_model(self, model, x0, n_steps=1000, renorm_interval=1):
        """
        Estimate Lyapunov exponent from a dynamical model.
        
        Args:
            model: Function that takes state and returns next state
            x0: Initial state
            n_steps: Number of time steps to simulate
            renorm_interval: How often to renormalize perturbation
        
        Returns:
            lyapunov_exponent: Largest Lyapunov exponent
            trajectory: List of (time, divergence) tuples
        """
        # Initialize
        x = np.array(x0, dtype=np.float64)
        
        # Random perturbation direction
        delta = np.random.randn(*x.shape)
        delta = delta / np.linalg.norm(delta) * self.perturbation_size
        
        # Track log divergence
        log_divergence = 0.0
        trajectory = []
        
        for step in range(n_steps):
            # Evolve both trajectories
            x_next = model(x)
            x_pert_next = model(x + delta)
            
            # Compute new perturbation
            delta_next = x_pert_next - x_next
            
            # Measure divergence
            divergence = np.linalg.norm(delta_next)
            
            # Accumulate log divergence
            if divergence > 0:
                log_divergence += np.log(divergence / self.perturbation_size)
            
            # Renormalize perturbation to prevent overflow
            if step % renorm_interval == 0 and divergence > 0:
                delta_next = delta_next / divergence * self.perturbation_size
            
            # Store trajectory
            trajectory.append((step * self.dt, log_divergence / ((step + 1) * self.dt)))
            
            # Update for next iteration
            x = x_next
            delta = delta_next
        
        # Average Lyapunov exponent
        lyapunov_exponent = log_divergence / (n_steps * self.dt)
        
        return lyapunov_exponent, trajectory

print("Lyapunov exponent estimator implemented!")

In [ ]:
# Create three example systems with different stability properties

# System 1: Stable (converging)
def stable_system(x):
    """Linear system with eigenvalues < 1 (stable)"""
    A = np.array([[0.9, 0.1], [-0.1, 0.85]])
    return A @ x + 0.01 * np.random.randn(2)

# System 2: Marginally stable (near critical)
def critical_system(x):
    """Nonlinear system near edge of chaos"""
    return np.array([
        0.99 * x[0] - 0.2 * x[1] + 0.1 * np.tanh(x[0]),
        0.2 * x[0] + 0.98 * x[1] + 0.1 * np.tanh(x[1])
    ])

# System 3: Chaotic (Lorenz-like)
def chaotic_system(x, sigma=10, rho=28, beta=8/3, dt=0.01):
    """Simplified Lorenz system (chaotic)"""
    # Using 2D projection for simplicity
    dx = np.array([
        sigma * (x[1] - x[0]),
        x[0] * (rho - x[1]) - x[1]
    ])
    return x + dt * dx

# Estimate Lyapunov exponents for each system
estimator = LyapunovExponentEstimator(dt=0.01)

systems = [
    ('Stable', stable_system, np.array([1.0, 1.0])),
    ('Critical', critical_system, np.array([1.0, 1.0])),
    ('Chaotic', chaotic_system, np.array([1.0, 1.0]))
]

results = {}
for name, system, x0 in systems:
    lyap, traj = estimator.estimate_from_model(system, x0, n_steps=1000)
    results[name] = {'lyapunov': lyap, 'trajectory': traj}
    print(f"{name} system: λ = {lyap:.4f}")

print("\nInterpretation:")
print("λ < 0: Stable (trajectories converge)")
print("λ ≈ 0: Critical (edge of chaos)")
print("λ > 0: Chaotic (trajectories diverge)")

In [ ]:
# Visualize Lyapunov exponent convergence
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, result) in enumerate(results.items()):
    ax = axes[idx]
    traj = result['trajectory']
    times = [t for t, _ in traj]
    lyaps = [l for _, l in traj]
    
    ax.plot(times, lyaps, linewidth=2)
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='λ=0 (critical)')
    ax.axhline(y=result['lyapunov'], color='green', linestyle='--', 
               alpha=0.5, label=f'Final: {result["lyapunov"]:.3f}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Lyapunov Exponent')
    ax.set_title(f'{name} System')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

print("\nNote: Lyapunov exponent should converge to a stable value.")
print("Oscillations indicate transient dynamics or insufficient sampling.")

## Part 3: Fixed Point Analysis

### What are Fixed Points?

A **fixed point** is a state where the dynamics stop changing:
```
x* = F(x*)  # System stays at x* forever
```

**Why fixed points matter**:
1. **Computational Attractors**: Fixed points represent stable computational states
2. **Memory**: Fixed points can store information (like attractor networks)
3. **Decision States**: Different fixed points can correspond to different decisions
4. **Stability Analysis**: Local dynamics around fixed points reveal stability

### Types of Fixed Points

Classified by eigenvalues of Jacobian at the fixed point:
- **Stable node**: All eigenvalues < 1 (attractive)
- **Unstable node**: All eigenvalues > 1 (repulsive)
- **Saddle point**: Mixed eigenvalues (attractive in some directions, repulsive in others)
- **Spiral**: Complex eigenvalues (oscillatory approach)

### Finding Fixed Points in Neural Networks

For RNNs, fixed points satisfy:
```
h* = f(W @ h* + b)
```
where f is the activation function (e.g., tanh, ReLU).

We can find these numerically using optimization or root-finding.

In [ ]:
class FixedPointFinder:
    """
    Find and analyze fixed points of a dynamical system.
    
    Fixed points satisfy: x* = F(x*)
    """
    
    def __init__(self, system, jacobian=None, tolerance=1e-6):
        """
        Args:
            system: Function that takes state and returns next state
            jacobian: Function that computes Jacobian (if None, use numerical)
            tolerance: Convergence tolerance
        """
        self.system = system
        self.jacobian = jacobian
        self.tolerance = tolerance
    
    def find_fixed_point(self, x0, max_iter=1000):
        """
        Find fixed point starting from initial guess x0.
        
        Solves: F(x) - x = 0
        
        Args:
            x0: Initial guess
            max_iter: Maximum iterations
        
        Returns:
            fixed_point: Solution (or None if not found)
            converged: Whether optimization converged
        """
        # Define equation: F(x) - x = 0
        def equation(x):
            return self.system(x) - x
        
        # Solve using scipy
        try:
            solution = fsolve(equation, x0, full_output=True, maxfev=max_iter)
            x_star, info, ier, msg = solution
            
            # Check convergence
            residual = np.linalg.norm(equation(x_star))
            converged = (ier == 1) and (residual < self.tolerance)
            
            if converged:
                return x_star, True
            else:
                return None, False
        except:
            return None, False
    
    def compute_jacobian(self, x, epsilon=1e-7):
        """
        Compute Jacobian numerically using finite differences.
        
        J[i,j] = ∂F_i/∂x_j
        """
        if self.jacobian is not None:
            return self.jacobian(x)
        
        n = len(x)
        J = np.zeros((n, n))
        
        for j in range(n):
            x_plus = x.copy()
            x_plus[j] += epsilon
            
            x_minus = x.copy()
            x_minus[j] -= epsilon
            
            J[:, j] = (self.system(x_plus) - self.system(x_minus)) / (2 * epsilon)
        
        return J
    
    def analyze_stability(self, fixed_point):
        """
        Analyze stability of fixed point.
        
        Computes eigenvalues of Jacobian at fixed point.
        
        Returns:
            Dictionary with:
            - eigenvalues: Eigenvalues of Jacobian
            - max_eigenvalue: Largest magnitude eigenvalue
            - stable: Whether fixed point is stable
            - type: Classification (stable/unstable/saddle)
        """
        # Compute Jacobian at fixed point
        J = self.compute_jacobian(fixed_point)
        
        # Eigenvalues
        eigvals = np.linalg.eigvals(J)
        max_eigval = np.max(np.abs(eigvals))
        
        # Stability
        stable = max_eigval < 1.0
        
        # Classification
        if stable:
            fp_type = 'stable'
        elif max_eigval > 1.0 and np.all(np.abs(eigvals) > 1.0):
            fp_type = 'unstable'
        else:
            fp_type = 'saddle'
        
        return {
            'eigenvalues': eigvals,
            'max_eigenvalue': max_eigval,
            'stable': stable,
            'type': fp_type,
            'jacobian': J
        }

print("Fixed point finder implemented!")

In [ ]:
# Define a simple 2D RNN-like system
def rnn_system(x):
    """
    Simple 2D recurrent system: h(t+1) = tanh(W @ h(t) + b)
    """
    W = np.array([[1.5, -1.0], [1.0, 0.5]])
    b = np.array([0.5, -0.5])
    return np.tanh(W @ x + b)

# Create fixed point finder
fp_finder = FixedPointFinder(rnn_system)

# Try multiple initial conditions to find different fixed points
initial_guesses = [
    np.array([0.0, 0.0]),
    np.array([1.0, 1.0]),
    np.array([-1.0, 1.0]),
    np.array([1.0, -1.0]),
    np.array([-1.0, -1.0]),
]

fixed_points = []
for x0 in initial_guesses:
    fp, converged = fp_finder.find_fixed_point(x0)
    if converged:
        # Check if this is a new fixed point (not duplicate)
        is_new = True
        for existing_fp in fixed_points:
            if np.linalg.norm(fp - existing_fp) < 0.1:
                is_new = False
                break
        
        if is_new:
            fixed_points.append(fp)

print(f"Found {len(fixed_points)} fixed point(s):\n")

# Analyze each fixed point
fp_analyses = []
for i, fp in enumerate(fixed_points):
    analysis = fp_finder.analyze_stability(fp)
    fp_analyses.append(analysis)
    
    print(f"Fixed Point {i+1}: [{fp[0]:.4f}, {fp[1]:.4f}]")
    print(f"  Type: {analysis['type']}")
    print(f"  Max |eigenvalue|: {analysis['max_eigenvalue']:.4f}")
    print(f"  Eigenvalues: {analysis['eigenvalues']}")
    print()

In [ ]:
# Visualize the vector field and fixed points
fig, ax = plt.subplots(figsize=(10, 8))

# Create grid for vector field
x_range = np.linspace(-2, 2, 20)
y_range = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(x_range, y_range)

# Compute vector field: F(x) - x
U = np.zeros_like(X)
V = np.zeros_like(Y)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        x = np.array([X[i, j], Y[i, j]])
        dx = rnn_system(x) - x
        U[i, j] = dx[0]
        V[i, j] = dx[1]

# Plot vector field
ax.quiver(X, Y, U, V, alpha=0.6, color='gray')

# Plot sample trajectories
for x0 in initial_guesses:
    trajectory = [x0]
    x = x0.copy()
    for _ in range(50):
        x = rnn_system(x)
        trajectory.append(x.copy())
    trajectory = np.array(trajectory)
    ax.plot(trajectory[:, 0], trajectory[:, 1], 'b-', alpha=0.3, linewidth=1)
    ax.plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=6)

# Plot fixed points
for i, (fp, analysis) in enumerate(zip(fixed_points, fp_analyses)):
    color = 'red' if analysis['stable'] else 'orange'
    marker = 'o' if analysis['type'] != 'saddle' else 's'
    ax.plot(fp[0], fp[1], marker=marker, color=color, 
            markersize=15, markeredgewidth=2, markeredgecolor='black',
            label=f"FP {i+1}: {analysis['type']}")

ax.set_xlabel('x₁')
ax.set_ylabel('x₂')
ax.set_title('RNN Dynamics: Vector Field and Fixed Points\n(Gray arrows show dynamics, blue lines show trajectories)')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Red circles: Stable fixed points (attractors)")
print("- Orange: Unstable or saddle points")
print("- Blue trajectories converge to stable fixed points")

## Part 4: Intrinsic Dimensionality Estimation

### What is Intrinsic Dimensionality?

**Intrinsic dimensionality** is the minimum number of parameters needed to describe the data.

Example: Points on a 2D plane embedded in 3D space have intrinsic dimension 2.

For neural networks:
- Hidden layer may have 1000 neurons (ambient dimension = 1000)
- But activations may lie on a lower-dimensional manifold (intrinsic dimension << 1000)

### Why This Matters

1. **Representational Efficiency**: Low intrinsic dimension means efficient encoding
2. **Generalization**: Lower dimension often correlates with better generalization
3. **Interpretability**: Fewer degrees of freedom are easier to understand
4. **Compression**: Reveals opportunity for dimensionality reduction

### Methods for Estimating Intrinsic Dimensionality

1. **PCA**: Count components explaining variance
2. **Participation Ratio**: Measure effective dimensionality from eigenvalues
3. **Local PCA**: Estimate dimension in local neighborhoods
4. **MLE Methods**: Maximum likelihood estimation of dimension

In [ ]:
class IntrinsicDimensionalityEstimator:
    """
    Estimate intrinsic dimensionality of neural representations.
    """
    
    def __init__(self):
        self.pca = None
    
    def participation_ratio(self, X):
        """
        Compute participation ratio (effective dimensionality).
        
        PR = (sum λ_i)² / sum λ_i²
        
        where λ_i are eigenvalues of covariance matrix.
        
        Args:
            X: Data matrix (n_samples, n_features)
        
        Returns:
            pr: Participation ratio
        """
        # Compute covariance
        X_centered = X - X.mean(axis=0)
        cov = (X_centered.T @ X_centered) / (len(X) - 1)
        
        # Eigenvalues
        eigvals = np.linalg.eigvalsh(cov)
        eigvals = np.maximum(eigvals, 0)  # Ensure non-negative
        
        # Participation ratio
        if np.sum(eigvals**2) == 0:
            return 0
        pr = np.sum(eigvals)**2 / np.sum(eigvals**2)
        
        return pr
    
    def explained_variance_dimension(self, X, variance_threshold=0.9):
        """
        Estimate dimension as number of PCs explaining threshold variance.
        
        Args:
            X: Data matrix (n_samples, n_features)
            variance_threshold: Cumulative variance threshold (e.g., 0.9 for 90%)
        
        Returns:
            n_components: Number of components
            explained_variance_ratio: Variance explained by each component
        """
        self.pca = PCA()
        self.pca.fit(X)
        
        cumsum = np.cumsum(self.pca.explained_variance_ratio_)
        n_components = np.argmax(cumsum >= variance_threshold) + 1
        
        return n_components, self.pca.explained_variance_ratio_
    
    def mle_dimension(self, X, k=10):
        """
        Maximum likelihood estimate of intrinsic dimension.
        
        Uses k-nearest neighbors method (Levina & Bickel, 2005).
        
        Args:
            X: Data matrix (n_samples, n_features)
            k: Number of nearest neighbors
        
        Returns:
            dimension: Estimated intrinsic dimension
        """
        n_samples = len(X)
        
        # Find k nearest neighbors
        nbrs = NearestNeighbors(n_neighbors=k+1).fit(X)
        distances, _ = nbrs.kneighbors(X)
        
        # Remove self (distance 0)
        distances = distances[:, 1:]
        
        # MLE formula
        # d = [ (k-1) / sum_i log(r_k(i) / r_1(i)) ]^{-1}
        r_k = distances[:, -1]  # k-th nearest neighbor distance
        r_1 = distances[:, 0]   # 1st nearest neighbor distance
        
        # Avoid log(0)
        ratio = r_k / (r_1 + 1e-10)
        ratio = np.maximum(ratio, 1e-10)
        
        dimension = (k - 1) / np.mean(np.log(ratio))
        
        return dimension
    
    def estimate_all(self, X, variance_threshold=0.9, k=10):
        """
        Run all dimensionality estimation methods.
        
        Returns:
            Dictionary with all estimates
        """
        pr = self.participation_ratio(X)
        pca_dim, var_ratios = self.explained_variance_dimension(X, variance_threshold)
        mle_dim = self.mle_dimension(X, k)
        
        return {
            'participation_ratio': pr,
            'pca_dimension': pca_dim,
            'mle_dimension': mle_dim,
            'explained_variance_ratios': var_ratios,
            'ambient_dimension': X.shape[1]
        }

print("Intrinsic dimensionality estimator implemented!")

In [ ]:
# Generate synthetic data with known intrinsic dimension
def generate_manifold_data(n_samples=1000, intrinsic_dim=3, ambient_dim=50, noise=0.1):
    """
    Generate data lying on a low-dimensional manifold in high-dimensional space.
    """
    # Generate intrinsic coordinates
    intrinsic_coords = np.random.randn(n_samples, intrinsic_dim)
    
    # Random projection to ambient space
    projection = np.random.randn(intrinsic_dim, ambient_dim)
    projection = projection / np.linalg.norm(projection, axis=0)
    
    # Project to high-dimensional space
    X = intrinsic_coords @ projection
    
    # Add noise
    X += noise * np.random.randn(n_samples, ambient_dim)
    
    return X, intrinsic_coords

# Generate data
true_dim = 5
X, intrinsic = generate_manifold_data(n_samples=1000, intrinsic_dim=true_dim, 
                                       ambient_dim=100, noise=0.05)

print(f"Generated data:")
print(f"  True intrinsic dimension: {true_dim}")
print(f"  Ambient dimension: {X.shape[1]}")
print(f"  Number of samples: {X.shape[0]}")

In [ ]:
# Estimate dimensionality
estimator = IntrinsicDimensionalityEstimator()
estimates = estimator.estimate_all(X, variance_threshold=0.95, k=10)

print("\nDimensionality Estimates:")
print("="*50)
print(f"Participation Ratio: {estimates['participation_ratio']:.2f}")
print(f"PCA Dimension (95% variance): {estimates['pca_dimension']}")
print(f"MLE Dimension: {estimates['mle_dimension']:.2f}")
print(f"\nTrue dimension: {true_dim}")
print(f"Ambient dimension: {estimates['ambient_dimension']}")

In [ ]:
# Visualize explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Scree plot (variance per component)
ax = axes[0]
var_ratios = estimates['explained_variance_ratios']
ax.bar(range(1, min(21, len(var_ratios)+1)), var_ratios[:20], alpha=0.7)
ax.axvline(x=true_dim, color='red', linestyle='--', linewidth=2, label=f'True dim: {true_dim}')
ax.axvline(x=estimates['pca_dimension'], color='green', linestyle='--', 
           linewidth=2, label=f'PCA dim: {estimates["pca_dimension"]}')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Explained Variance Ratio')
ax.set_title('Scree Plot: Variance per Component')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 2: Cumulative explained variance
ax = axes[1]
cumsum = np.cumsum(var_ratios[:20])
ax.plot(range(1, len(cumsum)+1), cumsum, 'b-', linewidth=2)
ax.axhline(y=0.95, color='gray', linestyle='--', alpha=0.5, label='95% threshold')
ax.axvline(x=true_dim, color='red', linestyle='--', linewidth=2, label=f'True dim: {true_dim}')
ax.axvline(x=estimates['pca_dimension'], color='green', linestyle='--', 
           linewidth=2, label=f'PCA dim: {estimates["pca_dimension"]}')
ax.set_xlabel('Number of Components')
ax.set_ylabel('Cumulative Explained Variance')
ax.set_title('Cumulative Explained Variance')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Sharp drop in variance indicates intrinsic dimensionality")
print("- All three methods should agree roughly with true dimension")
print("- Participation ratio gives 'effective' dimension (continuous value)")

## Part 5: Complete Dynamical Systems Analysis Pipeline

### Putting It All Together

Let's create a complete pipeline that analyzes an RNN's dynamics:

1. **Collect hidden state trajectories** from RNN
2. **DMD analysis**: Find dominant dynamic modes
3. **Lyapunov analysis**: Measure stability/chaos
4. **Fixed point analysis**: Find computational attractors
5. **Dimensionality analysis**: Measure complexity
6. **Generate comprehensive report**

In [ ]:
# Create a simple RNN for demonstration
class SimpleRNN(nn.Module):
    def __init__(self, input_size=10, hidden_size=50):
        super().__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
    
    def forward(self, x, h0=None):
        output, hn = self.rnn(x, h0)
        return output, hn

# Instantiate RNN
rnn = SimpleRNN(input_size=10, hidden_size=50)
rnn.eval()

# Generate input sequence
batch_size = 32
seq_length = 100
input_size = 10

# Random input sequence
x = torch.randn(batch_size, seq_length, input_size)

# Run RNN and collect hidden states
with torch.no_grad():
    outputs, _ = rnn(x)

# Extract hidden states: (batch, time, hidden_dim)
hidden_states = outputs.numpy()

print(f"Collected hidden states: {hidden_states.shape}")
print(f"  Batch size: {hidden_states.shape[0]}")
print(f"  Sequence length: {hidden_states.shape[1]}")
print(f"  Hidden dimension: {hidden_states.shape[2]}")

In [ ]:
class DynamicsAnalysisPipeline:
    """
    Complete pipeline for analyzing neural network dynamics.
    """
    
    def __init__(self):
        self.results = {}
    
    def analyze(self, hidden_states):
        """
        Run complete dynamical systems analysis.
        
        Args:
            hidden_states: Array of shape (batch, time, hidden_dim)
        
        Returns:
            Dictionary with all analysis results
        """
        print("Running Dynamical Systems Analysis...")
        print("="*50)
        
        # Reshape: (batch*time, hidden_dim)
        batch, time, hidden_dim = hidden_states.shape
        X_flat = hidden_states.reshape(-1, hidden_dim)
        
        # 1. Intrinsic Dimensionality
        print("\n1. Estimating intrinsic dimensionality...")
        dim_estimator = IntrinsicDimensionalityEstimator()
        dim_results = dim_estimator.estimate_all(X_flat)
        self.results['dimensionality'] = dim_results
        print(f"   Participation ratio: {dim_results['participation_ratio']:.2f}")
        print(f"   PCA dimension (95%): {dim_results['pca_dimension']}")
        
        # 2. DMD Analysis (use one trajectory)
        print("\n2. Running Dynamic Mode Decomposition...")
        X_traj = hidden_states[0].T  # (hidden_dim, time)
        dmd = DynamicModeDecomposition(rank=10)
        dmd.fit(X_traj)
        dmd_props = dmd.get_mode_properties()
        self.results['dmd'] = dmd_props
        n_stable = np.sum(dmd_props['growth_rates'] < 0)
        print(f"   Found {len(dmd_props['growth_rates'])} modes")
        print(f"   {n_stable} stable modes (growth rate < 0)")
        
        # 3. Simplified Lyapunov analysis
        print("\n3. Estimating stability (simplified)...")
        # Use jacobian-based approximation
        # Maximum eigenvalue of empirical transition matrix
        X1 = X_flat[:-1]
        X2 = X_flat[1:]
        # Approximate A such that X2 ≈ A @ X1
        A_approx = X2.T @ X1 @ np.linalg.pinv(X1.T @ X1)
        eigvals = np.linalg.eigvals(A_approx)
        max_eigval = np.max(np.abs(eigvals))
        self.results['stability'] = {
            'max_eigenvalue': max_eigval,
            'stable': max_eigval < 1.0
        }
        print(f"   Max eigenvalue: {max_eigval:.4f}")
        print(f"   System is {'stable' if max_eigval < 1.0 else 'unstable'}")
        
        print("\n" + "="*50)
        print("Analysis complete!")
        
        return self.results
    
    def plot_summary(self):
        """
        Create summary visualization of all analyses.
        """
        fig = plt.figure(figsize=(15, 10))
        gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)
        
        # Plot 1: Explained variance
        ax1 = fig.add_subplot(gs[0, 0])
        var_ratios = self.results['dimensionality']['explained_variance_ratios'][:20]
        ax1.bar(range(1, len(var_ratios)+1), var_ratios, alpha=0.7)
        ax1.set_xlabel('Component')
        ax1.set_ylabel('Explained Variance')
        ax1.set_title('PCA: Explained Variance')
        ax1.grid(True, alpha=0.3, axis='y')
        
        # Plot 2: DMD eigenvalues
        ax2 = fig.add_subplot(gs[0, 1])
        dmd = self.results['dmd']
        scatter = ax2.scatter(dmd['growth_rates'], dmd['frequencies'],
                            s=dmd['amplitudes']*500, c=dmd['amplitudes'],
                            cmap='viridis', alpha=0.6, edgecolors='black')
        ax2.axvline(x=0, color='red', linestyle='--', alpha=0.5)
        ax2.set_xlabel('Growth Rate')
        ax2.set_ylabel('Frequency (Hz)')
        ax2.set_title('DMD: Eigenvalue Spectrum')
        ax2.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax2, label='Amplitude')
        
        # Plot 3: Mode amplitudes
        ax3 = fig.add_subplot(gs[0, 2])
        colors = ['blue' if gr < 0 else 'red' for gr in dmd['growth_rates']]
        ax3.bar(range(len(dmd['amplitudes'])), dmd['amplitudes'], 
               color=colors, alpha=0.7)
        ax3.set_xlabel('Mode Index')
        ax3.set_ylabel('Amplitude')
        ax3.set_title('DMD: Mode Amplitudes')
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Plot 4: Summary statistics
        ax4 = fig.add_subplot(gs[1, :])
        ax4.axis('off')
        
        summary_text = f"""
        DYNAMICAL SYSTEMS ANALYSIS SUMMARY
        {'='*60}
        
        DIMENSIONALITY ANALYSIS:
        • Ambient dimension: {self.results['dimensionality']['ambient_dimension']}
        • Participation ratio: {self.results['dimensionality']['participation_ratio']:.2f}
        • PCA dimension (95% variance): {self.results['dimensionality']['pca_dimension']}
        • MLE dimension estimate: {self.results['dimensionality']['mle_dimension']:.2f}
        
        DYNAMIC MODE DECOMPOSITION:
        • Number of modes: {len(dmd['growth_rates'])}
        • Stable modes (growth < 0): {np.sum(dmd['growth_rates'] < 0)}
        • Unstable modes (growth > 0): {np.sum(dmd['growth_rates'] > 0)}
        • Dominant mode amplitude: {np.max(dmd['amplitudes']):.4f}
        
        STABILITY ANALYSIS:
        • Maximum eigenvalue: {self.results['stability']['max_eigenvalue']:.4f}
        • System classification: {'STABLE' if self.results['stability']['stable'] else 'UNSTABLE'}
        
        INTERPRETATION:
        • Low intrinsic dimension indicates efficient representation
        • Stable modes decay over time (robust to perturbations)
        • System stability suggests convergent dynamics
        """
        
        ax4.text(0.1, 0.5, summary_text, transform=ax4.transAxes,
                fontsize=10, verticalalignment='center',
                fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
        
        plt.tight_layout()
        plt.show()

print("Dynamics analysis pipeline implemented!")

In [ ]:
# Run complete analysis
pipeline = DynamicsAnalysisPipeline()
results = pipeline.analyze(hidden_states)

# Visualize results
pipeline.plot_summary()

## Summary and Next Steps

### What We Learned

1. **Dynamic Mode Decomposition**: Extract interpretable dynamic modes from neural trajectories
   - Find dominant patterns and oscillations
   - Predict future states
   - Identify stable/unstable dynamics

2. **Lyapunov Exponents**: Quantify chaos and stability
   - λ < 0: Stable, convergent dynamics
   - λ ≈ 0: Critical, edge-of-chaos
   - λ > 0: Chaotic, divergent dynamics

3. **Fixed Point Analysis**: Find computational attractors
   - Identify stable states
   - Characterize local stability
   - Understand decision states

4. **Intrinsic Dimensionality**: Measure representational complexity
   - Participation ratio for effective dimensionality
   - PCA for explained variance
   - MLE for local dimension estimates

### Key Takeaways

- **Neural networks are dynamical systems**: They have rich temporal structure
- **Low-dimensional structure is common**: High-dimensional activations often lie on low-dimensional manifolds
- **Stability matters**: Stable dynamics lead to robust, predictable computation
- **Edge of chaos is optimal**: Many successful networks operate near λ ≈ 0

### Applications to Your Research

1. **Recurrent Models**: Understand how RNNs/LSTMs process sequences
2. **Training Dynamics**: Track how representations evolve during learning
3. **Robustness Analysis**: Measure sensitivity to perturbations
4. **Model Comparison**: Compare dynamics across architectures
5. **Brain Alignment**: Match neural dynamics to brain recordings

### Recommended Next Steps

1. **Notebook 07**: Circuit extraction and latent RNN models
2. **Notebook 08**: Biophysical modeling with spiking networks
3. **Notebook 09**: Information theory and energy landscapes
4. **Notebook 10**: Advanced topics (meta-dynamics, topology, counterfactuals)

### Further Reading

- Kutz et al. (2016): *Dynamic Mode Decomposition*
- Sussillo & Barak (2013): *Opening the Black Box*
- Maheswaranathan et al. (2019): *Universality and individuality in neural dynamics*
- Chung & Abbott (2021): *Beyond the edge of chaos*
- Jazayeri & Afraz (2017): *Navigating the neural space*